In [1]:
import pandas as pd, numpy as np

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [5]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [8]:
locs=df[['category','icategory','tags','title','content']]

In [9]:
locs

,category,icategory,tags,title,content
0,aktualis,udvarhely,['Etéd'],"Elpusztult egy medve Etéd határában, orvvadász...",Sérült medve miatt riasztották a hatóságokat E...
1,aktualis,udvarhely,"['Hargita megye', 'Háromszék', 'Marosszék']",Célkeresztben a „medvetörvény” – a vadászok sz...,A többéves tiltás után tavaly ugyan megszülete...
2,aktualis,udvarhely,"['Háromszék', 'medve']","Tehetetlenek a lakók, az ablakból kénytelenek ...","Nemcsak éjszaka, hanem nappal is több medve já..."
3,aktualis,vasarhely,"['Felsőboldogfalva', 'medve']",Három juhot ölt meg a medve egy farcádi gazda ...,Három juhot ölt meg a medve a hétfőről keddre ...
4,aktualis,csik,"['medve', 'Tusnádfürdő']",Áthelyezik egy másik területre a Tusnádfürdőre...,A hatóságok kérésére át fogják helyezni egy má...
5,aktualis,123rf,"['Gyergyószék', 'medve', 'Udvarhelyszék']",Medvék miatt riasztották a hatóságokat és ébre...,"Vaslábon egy ház udvarára, míg a Székelyudvarh..."
6,aktualis,udvarhely,"['Kovászna megye', 'medve']",Külön megelőzési kvótát és a medvekárok kifize...,A környezetvédelmi minisztérium a közvitára bo...
7,vilag,csik,"['Belföld', 'medve']",A kormorán lekerülhet a védett fajok listájáró...,Képmutatásnak tartja Benkő Erika RMDSZ parlame...
8,aktualis,csik,"['medve', 'Tusnád']",Medve hatolt be egy tusnádfürdői portára,Egy tusnádfürdői portára ment be egy medve csü...
9,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Hárombocsos anyamedve bukkant fel Gyergyószent...,Gyergyószentmiklós közelébe „tévedt” medvék mi...


In [10]:
import json

In [11]:
lak=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [12]:
laks=[i for i in megyek.keys() if ((megyek[i]!='Más megye')and(i in lak.keys()))]

In [13]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [14]:
#look for perfect matches
decoded_locs={}
min_loc_len=5
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('5',j,k)

1 ["Etéd"] Etéd
2 Etéd
3 Etéd
5 tavaly Haraly
3 Zágon
5 számos Ádámos
1 ["Felsőboldogfalva", "medve"] Felsőboldogfalva
4 farcádi Farcád
1 ["medve", "Tusnádfürdő"] Tusnádfürdő
4 Tusnádfürdőre Tusnádfürdő
5 Tusnádfürdőre Tusnádfürdő
5 Vaslábon Vasláb
1 ["medve", "Tusnád"] Tusnád
4 tusnádfürdői Tusnádfürdő
5 tusnádfürdői Tusnádfürdő
1 ["Gyergyószentmiklós", "medve"] Gyergyószentmiklós
3 Gyergyószentmiklós
5 Gyergyószentmiklós Gyergyószentmiklós
3 Kőrispatak
5 Kőrispatak Kőrispatak
5 Kőrispatak Kispatak
1 ["Etéd", "medve"] Etéd


In [15]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(list(set(decoded_locs[locs.index[i]])))
        predicted_locs.append(dummy)
    else: 
        predicted_locs.append('')

In [16]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [18]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `data/szekelyhon_locs_curated` and manually verify predicted locations. Amend `actual_locs` where necessary.